In [2]:
import numpy as np
import pandas as pd
import re
from scipy.interpolate import SmoothBivariateSpline
import matplotlib.pyplot as plt 
import pyarrow 
from datetime import date

In [5]:
datos_estaciones=pd.read_csv('./data/NO2_estaciones.csv')
datos_estaciones.FECHA

0        2015-01-01 01:00:00
1        2015-01-01 02:00:00
2        2015-01-01 03:00:00
3        2015-01-01 04:00:00
4        2015-01-01 05:00:00
5        2015-01-01 06:00:00
6        2015-01-01 07:00:00
7        2015-01-01 08:00:00
8        2015-01-01 09:00:00
9        2015-01-01 10:00:00
10       2015-01-01 11:00:00
11       2015-01-01 12:00:00
12       2015-01-01 13:00:00
13       2015-01-01 14:00:00
14       2015-01-01 15:00:00
15       2015-01-01 16:00:00
16       2015-01-01 17:00:00
17       2015-01-01 18:00:00
18       2015-01-01 19:00:00
19       2015-01-01 20:00:00
20       2015-01-01 21:00:00
21       2015-01-01 22:00:00
22       2015-01-01 23:00:00
23       2015-01-02 00:00:00
24       2015-01-02 01:00:00
25       2015-01-02 02:00:00
26       2015-01-02 03:00:00
27       2015-01-02 04:00:00
28       2015-01-02 05:00:00
29       2015-01-02 06:00:00
                ...         
45930    2020-03-30 01:00:00
45931    2020-03-30 02:00:00
45932    2020-03-30 03:00:00
45933    2020-

In [15]:
%%time
mapa2016=pd.read_feather('./data/mapa_interpolado2016')
mapa2016.set_index(['fecha','longitud','latitud'],inplace=True)
mapa2016.sort_index(inplace=True)
mapa2016=mapa2016.unstack('longitud')

CPU times: user 1min 19s, sys: 18 s, total: 1min 37s
Wall time: 1min 39s


In [13]:
#Hacer un diccionario para cada ano-mes de las fechas de inicio y las fechas de fin
ano='2016'
mes='1'
pattern=ano+'-0'+mes+'-'
matching = [s for s in datos_estaciones.FECHA if pattern in s]
fecha_inicio=matching[0]
fecha_fin=matching[-1]
print(fecha_inicio,fecha_fin)

2016-01-01 00:00:00 2016-01-31 23:00:00


In [17]:
#se hace la media de los valores por mes
mes=mapa2016.query("fecha >= @fecha_inicio & fecha > @fecha_fin").mean(level='latitud')
mes

contaminacion                                                  \
longitud       3.677361    3.678361    3.679361    3.680361    3.681361   
latitud                                                                   
40.398103   -235.763931 -199.449295 -164.895355 -132.102112 -101.069550   
40.399103   -217.371246 -183.350571 -150.981339 -120.263550  -91.197220   
40.400103   -199.480804 -167.692902 -137.450089 -108.752342  -81.599670   
40.401103   -182.092621 -152.476334 -124.301620  -97.568481  -72.276909   
40.402103   -165.206680 -137.700836 -111.535942  -86.711967  -63.228939   
40.403103   -148.822998 -123.366432  -99.153038  -76.182808  -54.455753   
40.404103   -132.941574 -109.473106  -87.152908  -65.980995  -45.957352   
40.405103   -117.562393  -96.020851  -75.535568  -56.106529  -37.733738   
40.406103   -102.685471  -83.009689  -64.301003  -46.559410  -29.784912   
40.407103    -88.310791  -70.439606  -53.449219  -37.339642  -22.110872   
40.408103    -74.438370  -58.310596  -42.980213  -28.447222  -14.711618   
40.409103    -61.068203  -46.622673  -32.893990  -19.882149   -7.587151   
40.410103    -48.200287  -35.375832  -23.190546  -11.644424   -0.737470   
40.411103    -35.834621  -24.570072  -13.869882   -3.734050    5.837424   
40.412103    -23.971210  -14.205394   -4.931998    3.848977   12.137532   
40.413103    -12.610049   -4.281796    3.623106   11.104656   18.162853   
40.414103     -1.751142    5.200721   11.795430   18.032986   23.913387   
40.414444      1.836911    8.333410   14.494674   20.320702   25.811493   
40.415444     12.022298   17.224436   22.153690   26.810057   31.193539   
40.416444     21.705433   25.674381   29.429926   32.972065   36.300800   
40.417444     30.886316   33.683247   36.323380   38.806725   41.133270   
40.418444     39.564949   41.251030   42.834057   44.314034   45.690960   
40.419208     45.856979   46.735592   47.550278   48.301037   48.987865   
40.420208     53.649639   53.525307   53.385731   53.230904   53.060829   
40.421208     60.940044   59.873943   58.838402   57.833424   56.859005   
40.421564     63.414200   62.027592   60.687160   59.392910   58.144833   
40.422564     70.023552   67.778122   65.620781   63.551544   61.570400   
40.423564     76.130653   73.087570   70.171623   67.382828   64.721176   
40.423883     77.973152   74.688484   71.542816   68.536140   65.668465   
40.424883     83.417778   79.416145   75.588768   71.935654   68.456802   
...                 ...         ...         ...         ...         ...   
40.450547     51.301899   49.829769   48.455040   47.177715   45.997787   
40.451469     43.992432   43.360897   42.788830   42.276234   41.823105   
40.452469     35.581932   35.920891   36.275417   36.645515   37.031181   
40.453469     26.669178   28.039799   29.379223   30.687447   31.964470   
40.454469     17.254171   19.717628   22.100246   24.402029   26.622972   
40.455469      7.336913   10.954375   14.438493   17.789263   21.006689   
40.456469     -3.082598    1.750042    6.393958   10.849150   15.115618   
40.457469    -14.004360   -7.895373   -2.033357    3.581687    8.949761   
40.458469    -25.428375  -17.981869  -10.843452   -4.013123    2.509118   
40.459469    -37.354641  -28.509447  -20.036327  -11.935282   -4.206312   
40.460469    -49.783161  -39.478107  -29.611982  -20.184790  -11.196528   
40.461469    -62.713932  -50.887848  -39.570415  -28.761646  -18.461533   
40.462469    -76.146957  -62.738667  -49.911633  -37.665852  -26.001322   
40.463469    -90.082237  -75.030571  -60.635628  -46.897404  -33.815899   
40.464469   -104.519760  -87.763557  -71.742401  -56.456303  -41.905258   
40.465469   -119.459541 -100.937622  -83.231956  -66.342552  -50.269409   
40.465583   -121.194572 -102.467468  -84.566071  -67.490372  -51.240372   
40.466583   -136.693863 -116.132904  -96.482048  -77.741287  -59.910633   
40.467583   -152.695404 -130.239410 -108.780800  -88.319550  -68.855682   
40.468583   -169.199188 -144.787003

In [ ]:
plt.matshow(mes)
plt.show()

Para poder pasar los datos generados a una versión ordenada, que según Hadley Wickham, deberá realizar un "melt"

In [ ]:
mapa_interp=mapa.stack().reset_index()